## Tell cling where to find our mfem libraries

You may not even need this locally but for binder we do

In [ ]:
#pragma cling add_include_path("/home/jovyan/local/include")
#pragma cling add_library_path("/home/jovyan/local/lib")

## Load MFEM

Any non-default libraries must be loaded before you can `#include` files that use them

In [ ]:
#pragma cling load("libmfem")

## Now an example

In [ ]:
#include <fstream>
#include <iostream>
#include <sstream>

#include <mfem.hpp>
#include <glvis/xglvis.hpp>

In [ ]:
using namespace std;
using namespace mfem;

Mesh mesh = Mesh::MakeCartesian2D(5, 5, Element::TRIANGLE);
mesh.UniformRefinement();

H1_FECollection fec(2, mesh.Dimension());

FiniteElementSpace fespace(&mesh, &fec);
cout << "Number of finite element unknowns: " << fespace.GetTrueVSize() << endl;

Array<int> ess_tdof_list;
if (mesh.bdr_attributes.Size())
{
  Array<int> ess_bdr(mesh.bdr_attributes.Max());
  ess_bdr = 1;
  fespace.GetEssentialTrueDofs(ess_bdr, ess_tdof_list);
}

LinearForm b(&fespace);
ConstantCoefficient one(1.0);
b.AddDomainIntegrator(new DomainLFIntegrator(one));
b.Assemble();

GridFunction x(&fespace);
x = 0.0;

BilinearForm a(&fespace);
a.AddDomainIntegrator(new DiffusionIntegrator(one));
a.Assemble();

OperatorPtr A;
Vector B, X;
a.FormLinearSystem(ess_tdof_list, x, b, A, X, B);

cout << "Size of linear system: " << A->Height() << endl;

GSSmoother M((SparseMatrix&)(*A));
PCG(*A, M, B, X, 1, 200, 1e-12, 0.0);
a.RecoverFEMSolution(X, b, x);

In [ ]:
// for now we just get the stream as a string
std::stringstream ss;
ss << "solution\n" << mesh << x << flush;

In [ ]:
auto glv = glvis::glvis();
// the `+ "keys ...."' is optional
glv.show(ss.str() + "keys Rjml");
glv